In [87]:
import numpy as np
import math
import random
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
N = [] #best and second best x
B = [] #best x
sample_correct = [0] * 800 #correct at 10, 20, 50, 100, 200, 400, 600, 800
error_reward = [0]*800

delta = 0.05
sigma = 0.1 #noise

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1


class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        
class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1
        N.append(0)
        B.append(0)
                            

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        Y[y_index].pull_arm(reward, self.id)
        global N
        N[self.id] += 1
        return reward
    
    def find_worst_y(self):
        #print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = Y[i].lower_bound(self.arm_vector)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1
    R = [] #collection of reward
    A_id = [] #id of the i^th arm played
    times = 0
    theta_stimato = [0.0, 0.0, 0.0]

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y +=1
        
    def pull_arm(self, reward, id_arm):
        self.R.append(reward)
        self.A_id.append(id_arm)
        self.times += 1 
    
    def calcola_A(self):
        res = 0
        for i in range(0,self.times):
            #print("i", i)
            if i == 0:
                #print("i==0, print res", res)
                #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
                res = np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
                #print("res", res)
            else:
                #print("i!=0, print i, res", i, res)
                res += np.outer(np.transpose(A[self.A_id[i]].arm_vector), A[self.A_id[i]].arm_vector)
        #print("A", res)
        res += np.diag([1, 1, 1])
        return res

    def calcola_b(self):
        res = []
        global A
        for i in range(0,self.times):
            curr = A[self.A_id[i]].arm_vector
            curr = [self.R[i] * xx for xx in curr]
            if i == 0:
                res = curr
            else:
                res = [x + y for x, y in zip(res, curr)]
        return res
        
    def stima_theta(self):
        if (self.times == 0):
            return self.theta_stimato
        theta_s = np.dot(np.linalg.inv(self.calcola_A()), self.calcola_b())
        self.theta_stimato = theta_s
        return theta_s
    
    def media_reward(self, arm_vector):
        return np.dot(arm_vector, np.transpose(self.stima_theta()))
    
    def calcola_bound(self, arm_vector):
        ris = 2*sigma*math.sqrt(2)*norm_2_mat(arm_vector, np.linalg.inv(self.calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta))
        #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
        #print("bound", ris)
        return ris
    
    def upper_bound(self, arm_vector):
        return self.media_reward(arm_vector) + self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
    def lower_bound(self, arm_vector):
        return self.media_reward(arm_vector) - self.calcola_bound(arm_vector)
        #return self.media_reward(arm_vector)
    
def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def run():
    global t, A, Y
    t += 1
    #print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        curr = Y[A[i].worst_y].media_reward(A[i].arm_vector)
        #print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        #print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    #print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = Y[A[i].worst_y].upper_bound(A[i].arm_vector)
            if curr > maxi:
                second_best = i
                maxi = curr
    #print("second best", second_best, A[second_best].worst_y)
    B[best] += 1
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
        
        
        

In [88]:
a = arm( [0.7400797306398084, 0.576547832665651, 0.7349830663582422] )
b = arm( [0.345067231955826, 0.9156681583876762, 0.6823912524346282] )
c = arm( [0.5638993306859795, 0.8748338736453104, 0.18485633496104792] )
d = arm( [0.673719565597578, 0.1725870996908213, 0.7472676319688508] )
m = world( [[0.10491443939828526, 0.6793030919801211, 0.7194757379073787], [0.13546896362398142, 0.1217336458548296, 0.6991816425464136], [0.7898135475171382, 0.21428911809375328, 0.7623852971358278]] )
z = arm_y( [0.7233068388545373, 0.8898790946755905, 0.45671865937762546] )
k = arm_y( [0.36975365980964037, 0.5782920189315391, 0.3785203504783837] )
l = arm_y( [0.8700421843916122, 0.08522705600354452, 0.8195182497977863] )

In [89]:
best_x = -1
best_value = 0

for i in range(0, num_arms):
    for j in range(0, num_arms_y):
        curr = np.dot(np.dot(A[i].arm_vector, m.matrix), Y[j].arm_vector)
        print(i, j, curr)
        
for i in range(0, num_arms):
    min = 1000
    for j in range(0, num_arms_y):
        curr = np.dot(np.dot(A[i].arm_vector, m.matrix), Y[j].arm_vector)
        if curr < min:
            min = curr
    if min > best_value:
        best_value = min
        best_x = i

print(best_x)
print(best_value)

0 0 7.5904486631231265
0 1 7.469143503768381
0 2 10.08704974817524
1 0 12.947111505613869
1 1 13.695289708154004
1 2 18.06896826010647
2 0 5.859131915468996
2 1 6.62335243599327
2 2 7.714957076868007
3 0 5.143399827912187
3 1 5.900571363006973
3 2 7.089590477433962
1
12.947111505613869


In [90]:
A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

A[0].pull_arm(0)
A[0].pull_arm(1)
A[0].pull_arm(2)

A[1].pull_arm(0)
A[1].pull_arm(1)
A[1].pull_arm(2)

A[2].pull_arm(0)
A[2].pull_arm(1)
A[2].pull_arm(2)

A[3].pull_arm(0)
A[3].pull_arm(1)
A[3].pull_arm(2)

7.324186377867028

In [91]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)
A[3].find_worst_y()
print("x", 2, "y", A[0].worst_y)

x 0 y 0
x 1 y 0
x 2 y 0
x 2 y 0


In [92]:
res = np.dot(np.transpose(A[0].arm_vector), A[0].arm_vector)
res

0.6283323329248154

In [93]:
Y[0].calcola_bound(A[1].arm_vector)

0.362781182022823

In [94]:
run()

In [95]:
run()

In [96]:
print(Y[0].R)
print(Y[0].A_id)

[7.416486423956417, 7.548875722890885, 10.05704145619386, 12.917552462854268, 13.820753741476919, 18.039937236816865, 5.912117772122555, 6.751047401790297, 7.727649305120855, 7.728305856758168, 7.295453673538051, 10.06390151069482, 13.025908612007836, 13.804112841889628, 17.982542409649053, 5.933255896211729, 6.691052793427945, 7.654929588336634, 5.0623216219008516, 5.883570651302417, 7.324186377867028, 13.050358790772991, 7.67211124993065, 12.988598004967782, 7.650223423156941]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 1, 0, 1, 0]


In [97]:
for i in range(0, 800):
    run()
    if B.index(max(B)) == best_x:
        sample_correct[i] += 1
    error_reward[i] += best_value - Y[A[best_x].worst_y].media_reward(A[best_x].arm_vector)

In [98]:
Y[A[best_x].worst_y].media_reward(best_x)

array([6.77155953, 5.24410516, 6.12073048])

In [99]:
error_reward

[0.19927340438296604,
 -0.5784288758164919,
 -0.5238300091778605,
 -0.49419814407424134,
 -0.5430848151361971,
 -0.4920901775674391,
 -0.4454632949941537,
 -0.4089094155373161,
 -0.3650760363937273,
 -0.3230762591475944,
 -0.29036825567310487,
 -0.2685986173795474,
 -0.249450749093441,
 -0.2293291542925644,
 -0.21590373266294094,
 -0.20874701509160332,
 -0.19827671065158903,
 -0.1883865821689703,
 -0.1855318702882336,
 -0.1789498651352588,
 -0.16763780069863188,
 -0.15654498207194578,
 -0.14713558681268246,
 -0.13749805577341867,
 -0.13125712701796388,
 -0.130045682839377,
 -0.127158029055483,
 -0.12050128074291422,
 -0.11730543069035448,
 -0.11510868874818314,
 -0.11514369065835339,
 -0.1172758369923308,
 -0.11329994663136134,
 -0.10836499869297356,
 -0.11116459955307967,
 -0.10839362801831953,
 -0.10381793508224746,
 -0.10141170564893542,
 -0.09549235693177316,
 -0.09172311588991278,
 -0.08845714815022632,
 -0.08307506485066973,
 -0.0816642221491044,
 -0.07961433558926956,
 -0.073878

In [100]:
print(N)
print(B)
print(sample_correct)

[808, 808, 6, 3]
[0, 802, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [101]:
for i in range(0, num_arms_y):
    curr = Y[i].stima_theta()
    print("stima",curr)
    print("vero", np.dot(m.matrix, Y[i].arm_vector))

stima [6.77155953 5.24410516 6.12073048]
vero [8.42259865 7.27458946 2.7868485 ]
stima [6.30136619 5.29192218 5.99445618]
vero [8.75219623 4.98507618 5.1860856 ]
stima [6.30136619 5.29192218 5.99445618]
vero [12.78121275  9.16574983  3.53464086]


In [102]:
Y[0].R

[7.416486423956417,
 7.548875722890885,
 10.05704145619386,
 12.917552462854268,
 13.820753741476919,
 18.039937236816865,
 5.912117772122555,
 6.751047401790297,
 7.727649305120855,
 7.728305856758168,
 7.295453673538051,
 10.06390151069482,
 13.025908612007836,
 13.804112841889628,
 17.982542409649053,
 5.933255896211729,
 6.691052793427945,
 7.654929588336634,
 5.0623216219008516,
 5.883570651302417,
 7.324186377867028,
 13.050358790772991,
 7.67211124993065,
 12.988598004967782,
 7.650223423156941,
 13.029921983448899,
 7.537117226684344,
 12.869790965996378,
 7.460015756847027,
 12.761149404141083,
 7.63029364139609,
 12.830074700928089,
 7.527640763082034,
 12.931489096497831,
 7.535090742583905,
 12.91541516860382,
 7.60209438520754,
 12.847143692894797,
 7.654384711019851,
 12.94925611660269,
 7.555020820542777,
 13.038433170473013,
 7.599041183721331,
 12.942781813642734,
 7.660024783541196,
 12.92952685070368,
 7.687054361821466,
 13.083824188907075,
 7.607905719341277,
 12.9

In [103]:
A[0].find_worst_y()
print("x", 0, "y", A[0].worst_y)
A[1].find_worst_y()
print("x", 1, "y", A[0].worst_y)
A[2].find_worst_y()
print("x", 2, "y", A[0].worst_y)
A[3].find_worst_y()
print("x", 3, "y", A[0].worst_y)

x 0 y 1
x 1 y 1
x 2 y 1
x 3 y 1


In [104]:
import numpy as np        
f=open('074034TMG.txt','a')
np.savetxt(f, error_reward, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, sample_correct, fmt='%1.3f', newline=", ")
f.write("\n")
f.close()

In [105]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [106]:
print(N)
print(B)
print(sample_correct)

[808, 808, 6, 3]
[0, 802, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [107]:
for i in  range (0, num_arms):
    print(Y[A[i].worst_y].media_reward(A[i].arm_vector))

7.456388338688935
12.505166919537618
6.918193382261256
5.67155888302179
